# Consolidation Exericse 4 - Simulating Orbits

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Artemis_II_Rollout_%28NHQ202601170047%29_%28cropped%29.jpg/250px-Artemis_II_Rollout_%28NHQ202601170047%29_%28cropped%29.jpg" align="right"/>

In this consolidation exercise, you will investigate solving ordinary differential equations (ODEs).  In particular, you will look at is an _initial value problem_: given an initial state of a dynamic system, predict its behaviour.  This is the bedrock of numerical simulation in countless forms from robotics development to infesctious disease modelling.

You will draw on your experience of:
* Core Python syntax (calculations, conditionals, loops)
* Functions
* Using `numpy` for matrix maths
* Using `matplotlib` to produce plots
* Command line arguments

On completing this exercise, you will be able to use different numerical methods to solve an initial vaule problem for a system of ODEs, and comment on the issues involved.

## Task description

Write a program that compares three solutions to the same problem:
 - forward Euler integration,
 - the most common Runge-Kutta method,
 - the `solve_ivp` tool provided by `scipy` (exploring different solver options using a command line argument)

Your program should produce
1. A plot of the three trajectories from the respective solution methods
2. On-screen display of the errors, measured as the change in orbital energy from start to finish of the simulation
3. (Optional stretch goal) A plot of the variation in error with time step size for the Euler and Runge Kutta methods

Assumptions:
 - planar orbit, so only model motion in a 2-D plane, fixed to the centre of the Earth
 - mass of orbiter is negligible compared to Earth
 - simple Newtonian gravity for an orbit around a point-mass representation of Earth, assuming no other forces present besides Earth's gravity

Key data:
 - Gravitational constant $G = 6.674 \times 10^{-11}$ N(m/kg)$^2$ 
 - Mass of Earth $M_{\oplus} = 5.972 \times 10^{24}$ kg
 - Radius of Earth $R_{\oplus} = 6371$ km

Orbit data:
 - Initial altitude: $a_0 = 300$ km
 - Initial velocity: $0.9\sqrt{\frac{GM_{\oplus}}{R_{\oplus}+a_0}}$ tangential _i.e._ horizontal at starting point (apogee), or $6.96$ km/s.

Simulation specifications:
 - Duration: 90 minutes (approximately one orbit at 300km altitude)
 - Time step: 10s

### Example Outputs

#### Example 1

`python orbit_sim.py` with no command line argument should use the default `RK45` solver from `scipy`.  Errors are as follows:

```
Energy error from Euler: 6193181.396612663
Energy error from RK: -0.0077292099595069885
Energy error from scipy(RK45): -1066325.5488481969
```

#### Example 2

`python orbit_sim.py Radau` should select the `Radau` solver.  Errors are as follows:

```
Energy error from Euler: 6193181.396612663
Energy error from RK: -0.0077292099595069885
Energy error from scipy(Radau): -6448.816169783473
```

#### Example 3

`python orbit_sim_errors.py RK23` invokes my extended version including the error analysis.  It also selects the `RK23` solver.  Errors are as follows:

```
Energy error from Euler with timestep 0.1: 86499.8006708324
Energy error from RK with timestep 0.1: 7.078051567077637e-07
Energy error from Euler with timestep 0.3: 256989.40530384332
Energy error from RK with timestep 0.3: 3.725290298461914e-08
Energy error from Euler with timestep 1: 829337.0483363196
Energy error from RK with timestep 1: 7.450580596923828e-07
Energy error from Euler with timestep 3: 2293847.030715689
Energy error from RK with timestep 3: 1.5020370483398438e-05
Energy error from Euler with timestep 10: 6193181.396612663
Energy error from RK with timestep 10: 0.0077292099595069885
Energy error from Euler with timestep 30: 12756689.938549258
Energy error from RK with timestep 30: 2.844095431268215
Energy error from Euler with timestep 100: 22206908.869960353
Energy error from RK with timestep 100: 1308.9666029587388
Energy error from scipy(RK23): 151554.5655758679
```

## Background Information

### Orbital dynamics

The force acting on a body of mass $m$ orbiting the Earth is given by $\frac{GM_{\oplus}m}{r^2}$, where $r$ is the radius _i.e._ the distance to Earth's centre, acting towards the Earth's centre.

Since gravity is a conservative field, and we are not including any propulsive forces (_e.g._ rockets), the total energy of the system should be conserved.  The specific (_i.e._ per unit mass) energy of a body in Earth orbit is given by:

$ E = \frac{v^2}{2} - \frac{GM_{\oplus}}{r} $

where $v$ is the speed and $r$ is the distance from Earth centre _i.e._ radius.

### Conversion of ODE to first order state space model

Numerical methods can only handle first order differential equations, but can handle coupled equations in vector form.  This is often referred to as _state space modelling_.  For example, consider the following second order differential euqation:

$\ddot{p} + c\dot{p} + kp = 0$.

This can't be simulated in this form, but can be converted by introducing a second _state_ defined as the first derivative, $v=\dot{p}$.  Then the system can be re-written as:

$\dot{p} = v$

$\dot{v} = -cv -kp$

These are two first order differential equations, defining the derivative of each state as a function of all the states.

### Forward Euler integration

A computer cannot work in continuous time so has to look at the problem as a series of discrete time steps.  Assume a state space system in general form:

$ \dot{\mathbf{x}} = \mathbf{f}(\mathbf{x})$

where $\mathbf{x}$ is the state of the system and $\mathbf{f}$ is the model of its dynamics.  Then a simple way of predicting its behaviour over a time step is the _forward Euler approximation_:

$\mathbf{x}(t+\delta t) \approx \mathbf{x(t)} + \delta t \ \mathbf{f}\left( \mathbf{x}(t) \right) $ .

### Runge-Kutta Integration

A better approximation for many dynamic systems is given by the Runge Kutta method, which takes four different estimates of the gradient $k_1 \ldots k_4$ and combines them in something like a weighted average:

$k_1 = f \left( x(t) \right) $

$k_2 = f \left( x(t) + \frac{\delta t}{2}k_1 \right) $

$k_3 = f \left( x(t) + \frac{\delta t}{2}k_2 \right) $

$k_4 = f \left( x(t) + \delta t \ k_2 \right)$

$\mathbf{x}(t+\delta t) \approx \mathbf{x(t)} + \frac{\delta t}{6} \ \left( k_1 + 2k_2 + 2k_3 + k_4 \right) $

### Built-in `scipy` solvers

`scipy` provides a function `solve_ivp` for initial value problems.  You can read its documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp).